<a href="https://colab.research.google.com/github/danielbauer1860/LDS_Project/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from xml.etree import cElementTree as ET
import os
import pandas as pd

In [ ]:
aca_filepath = '/content/drive/MyDrive/Linguistic Data Science/data/2553/download/Texts/aca'
dem_filepath = '/content/drive/MyDrive/Linguistic Data Science/data/2553/download/Texts/dem'
fic_filepath = '/content/drive/MyDrive/Linguistic Data Science/data/2553/download/Texts/fic'
news_filepath = '/content/drive/MyDrive/Linguistic Data Science/data/2553/download/Texts/news'

filepaths = [aca_filepath, dem_filepath, fic_filepath, news_filepath]

In [ ]:
def extract_root(xml_file):
  tree = ET.parse(xml_file)
  return tree.getroot()

def extract_text(element):
    text = ""
    for child in element:
        if child.text:
            text += child.text + ""
        text += extract_text(child)
        if child.tail:
            text += child.tail + ""
    return text

In [ ]:
print(aca_filepath)

/content/drive/MyDrive/Linguistic Data Science/data/2553/download/Texts/aca


In [ ]:
def get_texts_from_directory(directory):
  texts = []
  for xml_file in os.listdir(directory):
    filename = os.fsdecode(xml_file)
    filepath = os.path.join(directory, filename)
    texts.append(extract_text(extract_root(filepath)))
  return texts

In [ ]:
texts =[]

for filepath in filepaths:
  category = os.path.basename(filepath).upper()
  directory_texts = get_texts_from_directory(filepath)
  for text in directory_texts:
    texts.append([text, category])

In [ ]:
def remove_initial_lines(df, column_name, amount):
    def remove_lines(text):
        return '\n'.join(text.split('\n')[amount:])

    df[column_name] = df[column_name].apply(remove_lines)
    return df

In [ ]:
df = pd.DataFrame(texts)
df.columns = ['text', 'category']
df

,text,category
0,Oxford Art Journal. Sample containing about ...,ACA
1,The Lancet. Sample containing about 44333 wo...,ACA
2,Computers and the humanities. Sample contain...,ACA
3,British journal of social work. Sample conta...,ACA
4,The British polity. Sample containing about ...,ACA
...,...,...
177,Liverpool Daily Post and Echo: Arts section....,NEWS
178,"The Guardian, electronic edition of 1989-12-...",NEWS
179,"Independent, electronic edition of 1989-10-0...",NEWS
180,The Scotsman: Applied Science pages. Sample ...,NEWS


In [ ]:
df = remove_initial_lines(df, 'text', 4)
df

,text,category
0,"Why do certain images matter to one, and why i...",ACA
1,"In many developing countries, the immunogenici...",ACA
2,Anthony Kenny\nThe British Library 1992\nSchol...,ACA
3,Beforehand he worked as a probation officer fo...,ACA
4,CONTINUITY AND CHANGE are features of every po...,ACA
...,...,...
177,"Artworks\nTHE Friends of Oriel Ynys Mon, Llang...",NEWS
178,THERE will be those who portray the report of ...,NEWS
179,BUCKINGHAM PALACE 30 September: The Princess R...,NEWS
180,The Scottish Landowners' Federation unequivoca...,NEWS


Replacing the new line tokens

In [ ]:
df['text'] = df['text'].str.replace("\n", " ")

In [ ]:
df.iloc[60]['text']

"Blood in her factory; another body found, and possibly identified. (The news had spread around in spite of police silence. No true blue inhabitant of Paradise Street or Mouncy Street ever needed to be told anything from the police; they had ways of finding out.) Rose put on more make-up and let her workers know it was business as usual. Under her firm gaze they bent to their machines. She had given Lily a holiday with pay, but Lily had obstinately reappeared and was working with the rest. Her face looked puffy. There was a truce between Rose and Gabriel, but none between Shirley and Gabriel because Gaby had discovered it to be Shirley who had leaked her secret to Rose. She had been searching Gaby's workroom, but she must have had her suspicions before. I suppose I'm easily read, thought Gabriel. In future she would be more careful. Discovery did not alter her plans, only slowed them down. Dagmar Blond moved about the factory as an unfriendly, enigmatic figure. This was the face that P

In [ ]:
output_directory = '/content/drive/MyDrive/Linguistic Data Science/data/bnc_baby_texts.csv'

df.to_csv(output_directory, sep= '|', index=False)